In [ ]:
#Refactoring Example
!sourcery review --fix APPS_3.5_W

In [ ]:
#Reordering Example1
import os
import ast
import hashlib
import astor

def get_sorted_hash(value):
    sorted_value = ''.join(sorted(value))
    return hashlib.sha256(sorted_value.encode()).hexdigest()

def transform_comparison(node, lines, file_path):
    if isinstance(node, ast.Compare):
        for op in node.ops:
            if isinstance(op, (ast.Gt, ast.Lt, ast.GtE, ast.LtE)):
                left_var = node.left.id if isinstance(node.left, ast.Name) else None
                right_var = node.comparators[0].id if isinstance(node.comparators[0], ast.Name) else None

                if left_var and right_var:
                    hash_left = get_sorted_hash(left_var)
                    hash_right = get_sorted_hash(right_var)

                    original_code = astor.to_source(node)

                    print(f"Comparing hashes: {left_var}: {hash_left}, {right_var}: {hash_right}")

                    if hash_left > hash_right:
                        if isinstance(op, ast.Gt):
                            node.left, node.comparators[0] = node.comparators[0], node.left
                        elif isinstance(op, ast.Lt):
                            node.left, node.comparators[0] = node.comparators[0], node.left
                            node.ops[0] = ast.Gt()
                        elif isinstance(op, ast.GtE):
                            node.left, node.comparators[0] = node.comparators[0], node.left
                            node.ops[0] = ast.LtE()
                        elif isinstance(op, ast.LtE):
                            node.left, node.comparators[0] = node.comparators[0], node.left
                            node.ops[0] = ast.GtE()

                        transformed_code = astor.to_source(node).strip()

                        if original_code.strip() != transformed_code:
                            transformed_line = lines[node.lineno - 1].replace(original_code.strip(), transformed_code)
                            print(f"Transformed in file: {file_path}")
                            print(f"Original line: {lines[node.lineno - 1].strip()}")
                            print(f"Transformed line: {transformed_code.strip()}\n")
                            return

def process_file(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
        lines = content.split('\n')

    tree = ast.parse(content)

    for node in ast.walk(tree):
        transform_comparison(node, lines, file_path)

    new_content = astor.to_source(tree)
    with open(file_path, 'w') as file:
        new_content = '\n'.join(line for line in new_content.split('\n') if line.strip())
        file.write(new_content)

def process_files(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith('.py'):
            file_path = os.path.join(folder_path, filename)
            try:
                process_file(file_path)
            except Exception as e:
                print(f"Error processing file {file_path}: {e}")

if __name__ == "__main__":
    folder_paths = [
        "APPS_3.5_W",
        "APPS_4_W"
    ]

    for folder_path in folder_paths:
        process_files(folder_path)


In [ ]:
#Reordering Example2
import os
import ast
import hashlib
import astor

def get_sorted_hash(value):
    sorted_value = ''.join(sorted(value))
    return hashlib.sha256(sorted_value.encode()).hexdigest()

def transform_operations(node, lines, file_path):
    if isinstance(node, ast.BinOp):
        if isinstance(node.op, (ast.Add, ast.Mult)):
            left_var = node.left.id if isinstance(node.left, ast.Name) else None
            right_var = node.right.id if isinstance(node.right, ast.Name) else None

            if left_var and right_var:
                hash_left = get_sorted_hash(left_var)
                hash_right = get_sorted_hash(right_var)

                original_code = astor.to_source(node)

                print(f"Comparing hashes: {left_var}: {hash_left}, {right_var}: {hash_right}")

                if hash_left > hash_right:
                    node.left, node.right = node.right, node.left

                    transformed_code = astor.to_source(node).strip()

                    if original_code.strip() != transformed_code:
                        transformed_line = lines[node.lineno - 1].replace(original_code.strip(), transformed_code)
                        print(f"Transformed in file: {file_path}")
                        print(f"Original line: {lines[node.lineno - 1].strip()}")
                        print(f"Transformed line: {transformed_code.strip()}\n")
                        return

def process_file(file_path):
    try:
        with open(file_path, 'r') as file:
            content = file.read()
            lines = content.split('\n')

        tree = ast.parse(content)

        for node in ast.walk(tree):
            transform_operations(node, lines, file_path)

        new_content = astor.to_source(tree)
        with open(file_path, 'w') as file:
            new_content = '\n'.join(line for line in new_content.split('\n') if line.strip())
            file.write(new_content)

    except Exception as e:
        print(f"Error processing file {file_path}: {e}")

def process_files(folder_paths):
    for folder_path in folder_paths:
        for filename in os.listdir(folder_path):
            if filename.endswith('.py'):
                file_path = os.path.join(folder_path, filename)
                process_file(file_path)

if __name__ == "__main__":
    folder_paths = [
        "APPS_3.5_W",
        "APPS_4_W"
    ]

    process_files(folder_paths)


In [ ]:
#Formating Examples
!autopep8 --in-place --select=E101,E11,E121,E122,E125 --verbose MBPP_3.5_W/*.py
!autopep8 --in-place --select=E126 --verbose MBPP_3.5_W/*.py
!autopep8 --in-place --select=E225,E226,E227,E228,E241,E242,E251,E252 --verbose MBPP_3.5_W/*.py
!autopep8 --in-place --select=E27 --verbose MBPP_3.5_W/*.py
!autopep8 --in-place --select=E26,E265 --verbose MBPP_3.5_W/*.py
!autopep8 --in-place --select=E266 --verbose MBPP_3.5_W/*.py
!autopep8 --in-place --select=E123 --verbose MBPP_3.5_W/*.py

In [ ]:
#Distortion Attack Example
!yapf -r MBPP_3.5_W
!sourcery review --fix APPS_3.5_W

In [ ]:
#Forgety Attack Example
!yapf -r MBPP_3.5_H
!sourcery review --fix APPS_3.5_H

In [ ]:
#Create Baseline
import shutil
import os

rq1_folder = ''
new_suffix = '_Baseline'
sub_folders = ['APPS_3.5', 'APPS_4', 'MBPP_3.5', 'MBPP_4', 'MBPP_Starcoder']


for sub_folder in sub_folders:
    sub_folder_path = os.path.join (rq1_folder, sub_folder)
    sub_sub_folders = ['H', 'W']

    for sub_sub_folder in sub_sub_folders:
        sub_sub_folder_path = os.path.join (sub_folder_path, f'{sub_folder}_{sub_sub_folder}')
        new_folder_name = f'{sub_folder}_{sub_sub_folder}{new_suffix}'
        new_folder_path = os.path.join (rq1_folder, sub_folder, new_folder_name)
        shutil.copytree (sub_sub_folder_path, new_folder_path)


In [ ]:
#Compute Results
# (After reimplementing watermarking)
import os
import filecmp

def calculate_accuracy(testing_folder, baseline_folder):
    test_files = [f for f in os.listdir(testing_folder) if f.endswith('.py')]
    baseline_files = [f for f in os.listdir(baseline_folder) if f.endswith('.py')]

    total_files = len(test_files)
    correct_count = 0

    for test_file in test_files:
        test_file_path = os.path.join(testing_folder, test_file)
        baseline_file = os.path.join(baseline_folder, test_file)

        if os.path.exists(baseline_file) and filecmp.cmp(test_file_path, baseline_file):
            correct_count += 1

    accuracy = (correct_count / total_files) * 100
    return accuracy

def calculate_accuracy_real(testing_folder, baseline_folder):
    test_files = [f for f in os.listdir(testing_folder) if f.endswith('.py')]
    baseline_files = [f for f in os.listdir(baseline_folder) if f.endswith('.py')]

    total_files = len(test_files)
    correct_count = 0

    for test_file in test_files:
        test_file_path = os.path.join(testing_folder, test_file)
        baseline_file = os.path.join(baseline_folder, test_file)

        if os.path.exists(baseline_file) and not filecmp.cmp(test_file_path, baseline_file):
            correct_count += 1

    accuracy = (correct_count / total_files) * 100
    return accuracy

# Define paths for testing and baseline folders for watermarked and real cases
watermarked_testing_folder = 'MBPP_3.5/MBPP_3.5_W'
watermarked_baseline_folder = 'MBPP_3.5/MBPP_3.5_W_Baseline'

real_testing_folder = 'MBPP_3.5/MBPP_3.5_H'
real_baseline_folder = 'MBPP_3.5/MBPP_3.5_H_Baseline'

# Calculate accuracy for watermarked and real cases
watermarked_accuracy = calculate_accuracy(watermarked_testing_folder, watermarked_baseline_folder)
real_accuracy = calculate_accuracy_real(real_testing_folder, real_baseline_folder)

# Calculate True Positive Rate (TPR)
total_watermarked_samples = len(os.listdir(watermarked_testing_folder))
true_positive = (watermarked_accuracy / 100) * total_watermarked_samples
false_negative = total_watermarked_samples - true_positive
tpr = true_positive / (true_positive + false_negative)

# Calculate False Positive Rate (FPR)
total_real_samples = len(os.listdir(real_testing_folder))
false_positive = ((total_real_samples - (real_accuracy / 100) * total_real_samples))
true_negative = total_real_samples - false_positive
fpr = false_positive / (false_positive + true_negative)

# Calculate Overall Accuracy
total_testing_samples = len(os.listdir(watermarked_testing_folder)) + len(os.listdir(real_testing_folder))
overall_correct_samples = watermarked_accuracy / 100 * len(os.listdir(watermarked_testing_folder)) + real_accuracy / 100 * len(os.listdir(real_testing_folder))
overall_accuracy = (overall_correct_samples / total_testing_samples) * 100

print('Overall Accuracy: {:.2f}%'.format(overall_accuracy))
print('True Positive Rate (TPR): {:.2f}%'.format(tpr * 100))
print('False Positive Rate (FPR): {:.2f}%'.format(fpr * 100))


In [ ]:
# Applicable Transformations Example1
#Sourcery
import re
import openpyxl

def extract_refactoring_info(file_content):
    lines = file_content.split('\n')
    matches = []

    for line in lines:
        if '[2K' in line and '/refactorings/' in line:
            file_path_start = line.find('[2K') + 3
            file_path_end = line.find('.py') + 3
            file_path = line[file_path_start:file_path_end]

            refactoring_start = line.find('/refactorings/') + len('/refactorings/')
            refactoring_end = line.find('/', refactoring_start)
            refactoring_type = line[refactoring_start:refactoring_end]

            matches.append((file_path, refactoring_type))

    return matches

def save_to_excel(results, output_file):
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = 'Refactoring Info'
    sheet['A1'] = 'File Path'
    sheet['B1'] = 'Refactoring Type'

    for row_num, result in enumerate(results, start=2):
        sheet.cell(row=row_num, column=1, value=result[0])
        sheet.cell(row=row_num, column=2, value=result[1])

    workbook.save(output_file)

if __name__ == "__main__":
    input_file_path = "Refactoring.rtf"
    output_file_path = "Refactoring_Info.xlsx"

    with open(input_file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()

    refactoring_info = extract_refactoring_info(file_content)
    save_to_excel(refactoring_info, output_file_path)

    print(f"Results saved to {output_file_path}")


In [ ]:
# Applicable Transformations Example2
import re
import openpyxl

def extract_refactoring_info(file_content):
    lines = file_content.split('\n')
    matches = []

    for line in lines:
        if 'Transformed in file:' in line:
            file_path_start = line.find('Transformed in file:') + len('Transformed in file:')
            file_path = line[file_path_start:].strip()
            file_path = file_path.rstrip('\\')  # Remove trailing backslashes
            matches.append((file_path, 'Reorder'))

    return matches

def save_to_excel(results, output_file):
    workbook = openpyxl.load_workbook(output_file)
    sheet = workbook.active

    for result in results:
        sheet.append(result)

    workbook.save(output_file)

if __name__ == "__main__":
    input_file_path = "Reorder.rtf"
    output_file_path = "Refactoring_Info.xlsx"

    with open(input_file_path, 'r', encoding='utf-8') as file:
        file_content = file.read()

    refactoring_info = extract_refactoring_info(file_content)
    save_to_excel(refactoring_info, output_file_path)

    print(f"Results appended to {output_file_path}")
